### Q. 어떤 요소가 대출금 상황 여부에 큰 영향을 줄까?

In [46]:
import pandas as pd
import os

In [47]:
os.chdir(r'C:\Users\hjb38\Documents\데이터 분석 과정\data\credit_data')

In [48]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
lb = pd.read_csv('loan_before.csv')

In [49]:
train.columns

Index(['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS',
       'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')

# 1. 문제정의

질문 1 : 어떤 요소가 대출금 상황 여부에 큰 영향을 줄까?

질문 2 : 그 요소들이 상환 여부에 어떤 영향을 줄까?

# 2. 방법론

## 2.1. 분석과정

이 질문에 대한 해답을 얻기 위해 아래와 '해석 가능한 머신러닝(xAI)'를 활용하였다.

### 2.1.1 프로세스

1단계 : feature engineering을 한다.<br>
2단계 : 머신러닝 모델을 만든다.<br>
3단계 : shap value를 통해 어떤 변수가 가장 큰 영향을 주는지 알아본다.<br>
4단계 : 영향을 가장 많이 주는 5개의 변수와 대출금 상환 여부와의 관계를 살펴본다.

## 2.2 Feature Engineering

### 2.2.1  AMT_CREDIT 에서 AMT_ANNUITY 을 나눠, 행당 사람이 몇 개월에 걸쳐 돈을 갚아야 하는지 변수화 한다.
- AMT_CREDIT : 총 대출금액
- AMT_ANNUITY : 매달 갚아야 하는 금액

In [50]:
train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_YEAR
0,162716,0,0.0,0,1,0,101250.0,480060.0,22509.0,Unaccompanied,...,-2256,NaN,2.0,15,Kindergarten,NaN,0.665732,NaN,-139.0,1.0
1,361317,0,0.0,0,1,2,225000.0,237024.0,12231.0,Unaccompanied,...,-4133,NaN,4.0,3,Business Entity Type 3,0.575699,0.535140,0.670652,-957.0,0.0
2,246925,0,1.0,0,0,0,171000.0,247675.5,26005.5,Unaccompanied,...,-4340,NaN,2.0,10,Business Entity Type 3,0.092273,0.002272,0.154744,-827.0,1.0
3,294331,0,1.0,1,0,0,225000.0,180000.0,9000.0,Unaccompanied,...,-1621,12.0,2.0,12,Trade: type 7,0.221815,0.120444,NaN,-569.0,2.0
4,152173,0,0.0,0,1,0,180000.0,337500.0,12852.0,Unaccompanied,...,-2870,NaN,1.0,12,XNA,NaN,0.722415,0.723837,-1544.0,4.0


In [51]:
# 몇 달동안 갚아야하는지 알기 위한 새로운 변수 만들기

train['AMT_CREDIT_TO_ANNUITY_RATIO']=train['AMT_CREDIT']/train['AMT_ANNUITY']
test['AMT_CREDIT_TO_ANNUITY_RATIO']=test['AMT_CREDIT']/test['AMT_ANNUITY']

In [52]:
lb.head()
# 같은 사람(SK_ID_CURR)이 여러개의 행에 걸쳐있다.
# -> 한 사람이 여러번의 대출을 받아서 값이 여러개 생긴 것.

,SK_ID_CURR,DAYS_CREDIT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,CREDIT_TYPE
0,400486,-2413,0,180000.0,Consumer credit
1,400486,-941,0,40500.0,Credit card
2,400486,-1110,0,114727.5,Consumer credit
3,400486,-2133,0,225000.0,Consumer credit
4,400486,-391,0,142290.0,Consumer credit


In [53]:
# SK_ID_CURR을 그룹핑 한 뒤 해당 ID인 사람이 받은 대출들의 평균을 알아보자

lb.groupby(['SK_ID_CURR'])['AMT_CREDIT_SUM'].mean().reset_index()

# reset_index() 를 통해 데이터프레임 형태로 변환

,SK_ID_CURR,AMT_CREDIT_SUM
0,100026,1.875000e+06
1,100041,6.325424e+04
2,100084,9.810000e+04
3,100088,3.555000e+05
4,100100,6.327487e+05
...,...,...
25707,456153,1.627073e+05
25708,456212,2.726224e+05
25709,456238,1.849170e+05
25710,456253,9.900000e+05


### 2.2.2 lb(이전 대출정보가 기록된 데이터)를 활용해 새 변수 생성

<strong>1. groupby 후 평균 내기</strong>(사람 id로 그룹핑 후 각 변수의 평균을 냄)
- AMT_CREDIT_SUM : 이전 대출의 금액
- DAYS_CREDIT : train, test로부터 며칠 전에 이전 대출을 진행했는지
- CNT_CREDIT_PROLONG : 대출연장을 한 횟수

<strong>2. groupby 후 횟수 확인</strong>(사람 id별로의 빈도수, 해당 사람이 이전에 대출을 몇 번 진행했는지 확인)

In [54]:
# train/test 데이터와 위 데이터 합치기 (SK_ID_CURR 컬럼을 기준으로)

train = pd.merge(train, lb.groupby(['SK_ID_CURR'])['AMT_CREDIT_SUM'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')
test = pd.merge(train, lb.groupby(['SK_ID_CURR'])['AMT_CREDIT_SUM'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')

In [55]:
# DAYS_CREDIT 과 CNT_CREDIT_PROLONG 컬럼도 평균을 계산해서 같이 넣어보자.

train = pd.merge(train, lb.groupby(['SK_ID_CURR'])['DAYS_CREDIT'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')
test = pd.merge(train, lb.groupby(['SK_ID_CURR'])['DAYS_CREDIT'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')

train = pd.merge(train, lb.groupby(['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')
test = pd.merge(train, lb.groupby(['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index(), 
                 on = 'SK_ID_CURR', how = 'left')

In [56]:
# 같은 SK_ID_CURR 를 가진 사람이 몇 번의 대출을 받았는가?

# groupby.size()를 하면 몇 개의 데이터를 그룹핑 했는지 보여준다.
# rename(columns={'':''})으로 컬럼의 이름도 바꿔준다.

# 이후 pd.merge로 합쳐주고 다시 train/test 데이터에 넣는다.

train = pd.merge(train, lb.groupby(['SK_ID_CURR']).size().reset_index().rename(columns={0:'count'}),
                on = 'SK_ID_CURR', how = 'left')

test = pd.merge(test, lb.groupby(['SK_ID_CURR']).size().reset_index().rename(columns={0:'count'}),
                on = 'SK_ID_CURR', how = 'left')

### 2.2.3 해석에 방해되는 불필요한 변수 제거

In [57]:
train.dtypes

SK_ID_CURR                       int64
TARGET                           int64
CODE_GENDER                    float64
FLAG_OWN_CAR                     int64
FLAG_OWN_REALTY                  int64
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
NAME_TYPE_SUITE                 object
NAME_INCOME_TYPE                object
NAME_EDUCATION_TYPE             object
NAME_HOUSING_TYPE               object
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED                  float64
DAYS_ID_PUBLISH                  int64
OWN_CAR_AGE                    float64
CNT_FAM_MEMBERS                float64
HOUR_APPR_PROCESS_START          int64
ORGANIZATION_TYPE               object
EXT_SOURCE_1                   float64
EXT_SOURCE_2                   float64
EXT_SOURCE_3                   float64
DAYS_LAST_PHONE_CHANGE         float64
AMT_REQ_CREDIT_BUREAU_YEA

In [58]:
# 범주형 데이터 및 겉보기엔 수치형이지만 그 의미는 범주형인 데이터(ex. 남자0, 여자1) 제거
# EXT_SOURCE_1/2/3 는 수치형이 맞지만, 데이터 자체에 대한 정보가 부족하므로 제거

del_list = ['CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'ORGANIZATION_TYPE', 
            'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

train = train.drop(del_list, axis=1)  # axis=1 은 컬럼, 0은 행을 뜻함
test = test.drop(del_list, axis=1)

In [59]:
train.dtypes

SK_ID_CURR                       int64
TARGET                           int64
FLAG_OWN_REALTY                  int64
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED                  float64
DAYS_ID_PUBLISH                  int64
OWN_CAR_AGE                    float64
CNT_FAM_MEMBERS                float64
HOUR_APPR_PROCESS_START          int64
DAYS_LAST_PHONE_CHANGE         float64
AMT_REQ_CREDIT_BUREAU_YEAR     float64
AMT_CREDIT_TO_ANNUITY_RATIO    float64
AMT_CREDIT_SUM                 float64
DAYS_CREDIT                    float64
CNT_CREDIT_PROLONG             float64
count                          float64
dtype: object